In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# comment this line if you want to run on GPU. 
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # -> exec on CPU

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import sciann as sn
from sciann.utils import sin, cos, mul

2022-01-05 12:32:17.727962: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-05 12:32:17.727989: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


---------------------- SCIANN 0.6.5.0 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.5.2 
Python Version: 3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0] 



In [8]:
from pathlib import Path
from turboflow.datasets.turb2D import Turb2D

time_idx = 256

def prepare_data(ds, num_data=64*64, random=True):

    data_dir = Path('/','home','dicarlo_d','Documents','Datasets','Turb2D.hdf5')
    tb = Turb2D(data_dir)
    tb.load_data(time_idx)
    
    xy = tb.xy # R x R x 2
    uv = tb.uv # R x R x 2
        
    uv = uv/np.max(np.abs(uv))
    assert np.min(uv) >= -1
    assert np.max(uv) <=  1
    
    XX = xy[::ds, ::ds, 0] # R x R
    YY = xy[::ds, ::ds, 1] # R x R
    UU = uv[::ds, ::ds, 0] # R x R
    VV = uv[::ds, ::ds, 1] # R x R
        
    R = XX.shape[0]
    N = R*R

    # Pick random data.
    if random:
        idx = np.random.choice(N, num_data, replace=False)
    else:
        idx = np.arange(0, N)
    
    x = XX.flatten()[idx,None] # RR x 1
    y = YY.flatten()[idx,None] # RR x 1
    
    u = UU.flatten()[idx,None] # RR x 1
    v = VV.flatten()[idx,None] # RR x 1
    
    return (x, y, u, v)

x_train, y_train, u_train, v_train = prepare_data(16, random=False)
print(x_train.shape)
x_test, y_test, u_test, v_test = prepare_data(4, random=False)
print(x_test.shape)

(256, 1)
(4096, 1)


In [12]:
x = sn.Variable('x', dtype='float64')
y = sn.Variable('y', dtype='float64')

nrff = 256
sigma = 10
# B = tf.Variable(tf.random.normal([1, nrff], dtype=tf.float64) * sigma, dtype=tf.float64, trainable=False)
B = np.random.normal(0,sigma,[1,nrff])

u = sn.Field('v', dtype='float64')
v = sn.Field('u', dtype='float64')

# u, v = sn.Functional([u, v], [sin(mul(x,B) + mul(y,B)), cos(mul(x,B) + mul(y,B))], 3*[256], 'tanh')
u, v = sn.Functional([u, v], [x, y], 3*[64], 'tanh')
d1 = sn.Data(u)
d2 = sn.Data(v)

u_x = sn.diff(u, x)
v_y = sn.diff(v, y)
c1 = sn.PDE(u_x + v_y)

In [13]:
model = sn.SciModel(
    inputs = [x, y],
    targets = [d1, d2, c1],
    loss_func= "mse",
    optimizer= "adam",
    plot_to_file='model.png',
)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  [(None, 1)]          0                                            
__________________________________________________________________________________________________
y (InputLayer)                  [(None, 1)]          0                                            
__________________________________________________________________________________________________
concat_3 (Concatenate)          (None, 2)            0           x[0][0]                          
                                                                 y[0][0]                          
__________________________________________________________________________________________________
D64b (Dense)                    (None, 64)           192         concat_3[0][0]             

In [ ]:
history = model.train(
    x_true=[x_train, y_train],
    y_true=[u_train, v_train, 'zeros'],
    batch_size=1024,
    learning_rate=0.002,
    epochs=5000,
    verbose=0)


Total samples: 256 
Batch size: 256 
Total batches: 1 



In [ ]:
plt.semilogy(history.history['loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

In [ ]:
print(history)

In [ ]:
pred = model.predict([x_test, y_test])

In [ ]:
u_pred = pred[0]
plt.imshow(u_pred.reshape(256,256))
